In [18]:
import librosa
import os
import pandas
import soundfile
import numpy as np
import pandas as pd
from IPython.display import clear_output
import tensorflow as tf

In [17]:
pip install tensorflow

You should consider upgrading via the 'C:\Users\Jonat\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.



  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached libclang-13.0.0-py2.py3-none-win_amd64.whl (13.9 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Markdown-3.3.6-py3-none-any.whl (97 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.

In [2]:
# pass this function a file and it will return a list of features.
def get_features(file):
    with soundfile.SoundFile(file) as sound_file:
        result = np.array([])
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        stft = np.abs(librosa.stft(X))
        
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
        
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
        
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
        
    return result

In [3]:
emotions = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

In [36]:
directory = "data_16khz"
X,y = [],[]
total = 0
for dir,sub_dir,files in os.walk(directory):
    for file in files:
        emotion = file.split("-")[2]
        if emotion not in emotions: continue
        total += 1
        print("total files added :",total)
        url = dir+"\\"+file
        y.append(emotions[emotion])
        X.append(get_features(url))
        clear_output(wait=True)

total files added : 99


<ipython-input-2-b28ec5c1fe3d>:15: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.0517578e-05
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


KeyboardInterrupt: 

In [38]:
y = tf.keras.utils.to_categorical(y, num_classes=8)
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, random_state=23)

ValueError: invalid literal for int() with base 10: 'neutral'

In [23]:
hidden_layer_count = 1
nodes_per_layer = [300,150]
input_features = len(X_train[0])
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=nodes_per_layer[0], input_dim=input_features, activation="relu"))
model.add(tf.keras.layers.Dense(units=nodes_per_layer[1], activation="relu"))
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 300)               54300     
                                                                 
 dense_6 (Dense)             (None, 150)               45150     
                                                                 
 dense_7 (Dense)             (None, 1)                 151       
                                                                 
Total params: 99,601
Trainable params: 99,601
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = model.fit(X_train,y_train,epochs=100)
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'str'>"})

In [34]:
types = []
for x in y_train :
    for item in x:
        types.append(type(item))

In [35]:
set(types)

{str}

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'fearful',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'surprised',
 'surprised',
 'surprised',
 'surprised',
 'surprised',
 'surprised',
 'surprised',
 'surprised',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'happy',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'fearfu